In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  

In [61]:
pair = "USDCHF"
NB_H1_CANDLES = 10000

In [62]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [63]:
rates = Rates(pair, 13*2000, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [64]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2021-08-12 00:00:00,0.92165,0.92177,0.92095,0.92106,0.92106,0.92165,13.414634,85.365854,71.951220,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-08-12 01:00:00,0.92095,0.92164,0.92095,0.92152,0.92095,0.92152,0.000000,82.608696,82.608696,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-08-12 02:00:00,0.92153,0.92163,0.92138,0.92147,0.92147,0.92153,36.000000,60.000000,24.000000,...,71.951220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-08-12 03:00:00,0.92147,0.92181,0.92130,0.92156,0.92147,0.92156,33.333333,50.980392,17.647059,...,82.608696,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2021-08-12 04:00:00,0.92156,0.92174,0.92140,0.92155,0.92155,0.92156,44.117647,47.058824,2.941176,...,24.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2021-08-12 00:00:00,0.92165,0.92177,0.92095,0.92106,0.92106,0.92165,13.414634,85.365854,71.951220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-12 01:00:00,0.92095,0.92164,0.92095,0.92152,0.92095,0.92152,0.000000,82.608696,82.608696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-12 02:00:00,0.92153,0.92163,0.92138,0.92147,0.92147,0.92153,36.000000,60.000000,24.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-12 03:00:00,0.92147,0.92181,0.92130,0.92156,0.92147,0.92156,33.333333,50.980392,17.647059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-12 04:00:00,0.92156,0.92174,0.92140,0.92155,0.92155,0.92156,44.117647,47.058824,2.941176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2023-03-22 19:00:00,0.92322,0.92425,0.92271,0.92381,0.92322,0.92381,33.116883,71.428571,38.311688,...,0.922624,0.923793,0.921455,0.001558,0.923108,0.926510,0.919706,-0.000298,-0.000607,0.000310
9995,2023-03-22 20:00:00,0.92381,0.92381,0.91470,0.91714,0.91714,0.92381,26.783754,100.000000,73.216246,...,0.922447,0.924608,0.920285,0.002171,0.922539,0.927391,0.917687,-0.000745,-0.000635,-0.000110
9996,2023-03-22 21:00:00,0.91715,0.91924,0.91608,0.91793,0.91715,0.91793,33.860759,58.544304,24.683544,...,0.922244,0.925253,0.919235,0.002359,0.922100,0.927196,0.917004,-0.001024,-0.000713,-0.000311
9997,2023-03-22 22:00:00,0.91793,0.91798,0.91722,0.91732,0.91732,0.91793,13.157895,93.421053,80.263158,...,0.922021,0.925702,0.918339,0.002334,0.921645,0.926545,0.916745,-0.001279,-0.000826,-0.000453


In [66]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00023

In [67]:
df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

C:\Users\meysam\AppData\Local\Temp\ipykernel_2688\3875528746.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
C:\Users\meysam\AppData\Local\Temp\ipykernel_2688\3875528746.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)


,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST,low_perv,high_perv,low_next,high_next
0,2021-08-13 09:00:00,0.92313,0.92359,0.92304,0.92352,0.92313,0.92352,16.363636,87.272727,70.909091,...,0.922921,0.923563,0.922279,0.000219,0.000225,-0.000006,0.92295,0.92313,0.92270,0.92374
1,2021-08-13 10:00:00,0.92352,0.92374,0.92270,0.92296,0.92296,0.92352,25.000000,78.846154,53.846154,...,0.922925,0.923705,0.922145,0.000194,0.000218,-0.000024,0.92304,0.92359,0.92235,0.92303
2,2021-08-13 11:00:00,0.92296,0.92303,0.92235,0.92240,0.92240,0.92296,7.352941,89.705882,82.352941,...,0.922874,0.923718,0.922030,0.000131,0.000199,-0.000068,0.92270,0.92374,0.92176,0.92256
3,2021-08-13 12:00:00,0.92239,0.92256,0.92176,0.92187,0.92187,0.92239,13.750000,78.750000,65.000000,...,0.922776,0.923756,0.921796,0.000039,0.000165,-0.000125,0.92235,0.92303,0.92151,0.92209
4,2021-08-13 13:00:00,0.92187,0.92209,0.92151,0.92176,0.92176,0.92187,43.103448,62.068966,18.965517,...,0.922677,0.923699,0.921655,-0.000041,0.000121,-0.000162,0.92176,0.92256,0.92045,0.92177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9960,2023-03-22 18:00:00,0.92368,0.92405,0.92264,0.92321,0.92321,0.92368,40.425532,73.758865,33.333333,...,0.923034,0.926656,0.919412,-0.000402,-0.000685,0.000283,0.92197,0.92405,0.92271,0.92425
9961,2023-03-22 19:00:00,0.92322,0.92425,0.92271,0.92381,0.92322,0.92381,33.116883,71.428571,38.311688,...,0.923108,0.926510,0.919706,-0.000298,-0.000607,0.000310,0.92264,0.92405,0.91470,0.92381
9962,2023-03-22 20:00:00,0.92381,0.92381,0.91470,0.91714,0.91714,0.92381,26.783754,100.000000,73.216246,...,0.922539,0.927391,0.917687,-0.000745,-0.000635,-0.000110,0.92271,0.92425,0.91608,0.91924
9963,2023-03-22 21:00:00,0.91715,0.91924,0.91608,0.91793,0.91715,0.91793,33.860759,58.544304,24.683544,...,0.922100,0.927196,0.917004,-0.001024,-0.000713,-0.000311,0.91470,0.92381,0.91722,0.91798


In [68]:
BUY = 1
SELL = -1
NONE = 0

df_an['BUY_SIG'] = 0
df_an['SELL_SIG'] = 0
df_an['SIGNAL'] = 0
df_an['SIGNAL_LABEL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

In [69]:
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.BUY_SIG[i] = 1
                    df_an.SIGNAL[i] = BUY
                    df_an.SIGNAL_LABEL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SELL_SIG[i] = 1
                    df_an.SIGNAL[i] = SELL
                    df_an.SIGNAL_LABEL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_2688\2220319849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SELL_SIG[i] = 1
C:\Users\meysam\AppData\Local\Temp\ipykernel_2688\2220319849.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_2688\2220319849.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL_LABEL[i] = SELL
C:\Users\meysam\AppData

In [70]:
df_an.SIGNAL

0       0
1       0
2       0
3       0
4      -1
       ..
9960    0
9961    0
9962    0
9963    0
9964    0
Name: SIGNAL, Length: 9965, dtype: int64

In [71]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'low_perv', 'high_perv', 'low_next', 'high_next',
       'BUY_SIG', 'SELL_SIG', 'SIGNAL', 'SIGNAL_LABEL', 'TP', 'SL'],
      dtype='object')

In [108]:
df_features = df_an[feature_col]
mu = df_features.mean()
std = df_features.std()
df_features = (df_features-mu)/std

In [110]:
df_features.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9965 entries, 0 to 9964
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   open              9965 non-null   float64
 1   high              9965 non-null   float64
 2   low               9965 non-null   float64
 3   close             9965 non-null   float64
 4   RSI_14            9965 non-null   float64
 5   ATR_14            9965 non-null   float64
 6   BB_MA             9965 non-null   float64
 7   BB_UP             9965 non-null   float64
 8   BB_LW             9965 non-null   float64
 9   EMA               9965 non-null   float64
 10  KeUp              9965 non-null   float64
 11  KeLo              9965 non-null   float64
 12  MACD              9965 non-null   float64
 13  SIGNAL_MACD       9965 non-null   float64
 14  HIST              9965 non-null   float64
 15  body_lower        9965 non-null   float64
 16  body_upper        9965 non-null   float64


In [111]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
#     'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
for i in range(4,len(df_features)):
    feature = []
    for fc in feature_col:
        feature.append(df_features[fc][i])
        
    for fc in feature_col:
        feature.append(df_features[fc][i-1])
        feature.append(df_features[fc][i-2])
        feature.append(df_features[fc][i-3])
        feature.append(df_features[fc][i-4])
        
    X.append(feature)
    Y.append(df_an.SIGNAL[i])

In [112]:
len(feature)

190

In [116]:
X_train = X[:8000]
Y_train = Y[:8000]
X_test = X[8000:]
Y_test = Y[8000:]


In [114]:
# X_train = np.array(X_train)

In [115]:
# X_train = (X_train - X_train.mean(axis=0))/X_train.std(axis=0)

In [80]:
# X_train_extend = []
# Y_train_extend = []
# for i in range(8000):
#     if Y_train[i] != NONE:
#         for j in range(22):
#             X_train_extend.append(X[i])
#             Y_train_extend.append(Y[j])
#     X_train_extend.append(X[i])
#     Y_train_extend.append(Y[j])

In [81]:
p = np.random.permutation(len(X_train_extend))

In [82]:
# X_train_extend = np.array(X_train_extend)
# Y_train_extend = np.array(Y_train_extend)

In [83]:
# X_train_extend = X_train_extend[p]
# Y_train_extend = Y_train_extend[p]

In [84]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [85]:
df_an.SIGNAL.value_counts()

 0    9123
-1     441
 1     401
Name: SIGNAL, dtype: int64

In [55]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [119]:
clf = MLPClassifier( solver='sgd', alpha=1e-5,learning_rate_init=1e-5,
                     tol=0.0000010, hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [ ]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 1.40677331
Iteration 2, loss = 1.26146663
Iteration 3, loss = 1.12948421
Iteration 4, loss = 1.01802467
Iteration 5, loss = 0.92595519
Iteration 6, loss = 0.84982518
Iteration 7, loss = 0.78773586
Iteration 8, loss = 0.73657084
Iteration 9, loss = 0.69421822
Iteration 10, loss = 0.65888064
Iteration 11, loss = 0.62905192
Iteration 12, loss = 0.60368965
Iteration 13, loss = 0.58189005
Iteration 14, loss = 0.56305531
Iteration 15, loss = 0.54663891
Iteration 16, loss = 0.53218036
Iteration 17, loss = 0.51951833
Iteration 18, loss = 0.50824822
Iteration 19, loss = 0.49820089
Iteration 20, loss = 0.48920264
Iteration 21, loss = 0.48117367
Iteration 22, loss = 0.47390023
Iteration 23, loss = 0.46733974
Iteration 24, loss = 0.46139407
Iteration 25, loss = 0.45595880
Iteration 26, loss = 0.45104429
Iteration 27, loss = 0.44652745
Iteration 28, loss = 0.44238063
Iteration 29, loss = 0.43859376
Iteration 30, loss = 0.43512391
Iteration 31, loss = 0.43188212
Iteration 32, los

Iteration 253, loss = 0.36977839
Iteration 254, loss = 0.36972935
Iteration 255, loss = 0.36968261
Iteration 256, loss = 0.36963288
Iteration 257, loss = 0.36958340
Iteration 258, loss = 0.36953775
Iteration 259, loss = 0.36948614
Iteration 260, loss = 0.36943795
Iteration 261, loss = 0.36939236
Iteration 262, loss = 0.36934298
Iteration 263, loss = 0.36929379
Iteration 264, loss = 0.36924727
Iteration 265, loss = 0.36920046
Iteration 266, loss = 0.36915071
Iteration 267, loss = 0.36910502
Iteration 268, loss = 0.36905506
Iteration 269, loss = 0.36901478
Iteration 270, loss = 0.36896015
Iteration 271, loss = 0.36891289
Iteration 272, loss = 0.36886865
Iteration 273, loss = 0.36882069
Iteration 274, loss = 0.36877514
Iteration 275, loss = 0.36872752
Iteration 276, loss = 0.36868055
Iteration 277, loss = 0.36863125
Iteration 278, loss = 0.36858721
Iteration 279, loss = 0.36853893
Iteration 280, loss = 0.36849448
Iteration 281, loss = 0.36844699
Iteration 282, loss = 0.36840077
Iteration 

Iteration 502, loss = 0.35989038
Iteration 503, loss = 0.35985751
Iteration 504, loss = 0.35982622
Iteration 505, loss = 0.35979293
Iteration 506, loss = 0.35976016
Iteration 507, loss = 0.35973106
Iteration 508, loss = 0.35969512
Iteration 509, loss = 0.35966190
Iteration 510, loss = 0.35963054
Iteration 511, loss = 0.35959829
Iteration 512, loss = 0.35956667
Iteration 513, loss = 0.35953390
Iteration 514, loss = 0.35950073
Iteration 515, loss = 0.35946893
Iteration 516, loss = 0.35943843
Iteration 517, loss = 0.35940547
Iteration 518, loss = 0.35937407
Iteration 519, loss = 0.35934023
Iteration 520, loss = 0.35930952
Iteration 521, loss = 0.35927648
Iteration 522, loss = 0.35924597
Iteration 523, loss = 0.35921660
Iteration 524, loss = 0.35918003
Iteration 525, loss = 0.35915092
Iteration 526, loss = 0.35911663
Iteration 527, loss = 0.35908676
Iteration 528, loss = 0.35905414
Iteration 529, loss = 0.35902232
Iteration 530, loss = 0.35899023
Iteration 531, loss = 0.35895936
Iteration 

Iteration 751, loss = 0.35271964
Iteration 752, loss = 0.35269494
Iteration 753, loss = 0.35266881
Iteration 754, loss = 0.35264352
Iteration 755, loss = 0.35261910
Iteration 756, loss = 0.35259340
Iteration 757, loss = 0.35256841
Iteration 758, loss = 0.35254296
Iteration 759, loss = 0.35251945
Iteration 760, loss = 0.35249162
Iteration 761, loss = 0.35246584
Iteration 762, loss = 0.35244099
Iteration 763, loss = 0.35241655
Iteration 764, loss = 0.35239012
Iteration 765, loss = 0.35236566
Iteration 766, loss = 0.35233997
Iteration 767, loss = 0.35231654
Iteration 768, loss = 0.35228920
Iteration 769, loss = 0.35226745
Iteration 770, loss = 0.35224287
Iteration 771, loss = 0.35221421
Iteration 772, loss = 0.35218848
Iteration 773, loss = 0.35216401
Iteration 774, loss = 0.35213953
Iteration 775, loss = 0.35211563
Iteration 776, loss = 0.35208880
Iteration 777, loss = 0.35206516
Iteration 778, loss = 0.35204015
Iteration 779, loss = 0.35201596
Iteration 780, loss = 0.35198906
Iteration 

Iteration 1000, loss = 0.34697656
Iteration 1001, loss = 0.34695463
Iteration 1002, loss = 0.34693441
Iteration 1003, loss = 0.34691409
Iteration 1004, loss = 0.34689355
Iteration 1005, loss = 0.34687156
Iteration 1006, loss = 0.34685167
Iteration 1007, loss = 0.34682977
Iteration 1008, loss = 0.34680833
Iteration 1009, loss = 0.34679067
Iteration 1010, loss = 0.34677122
Iteration 1011, loss = 0.34674682
Iteration 1012, loss = 0.34672596
Iteration 1013, loss = 0.34670953
Iteration 1014, loss = 0.34668823
Iteration 1015, loss = 0.34666411
Iteration 1016, loss = 0.34664241
Iteration 1017, loss = 0.34662286
Iteration 1018, loss = 0.34660177
Iteration 1019, loss = 0.34658117
Iteration 1020, loss = 0.34656358
Iteration 1021, loss = 0.34654508
Iteration 1022, loss = 0.34651910
Iteration 1023, loss = 0.34649954
Iteration 1024, loss = 0.34647881
Iteration 1025, loss = 0.34646053
Iteration 1026, loss = 0.34643790
Iteration 1027, loss = 0.34641848
Iteration 1028, loss = 0.34639518
Iteration 1029

Iteration 1241, loss = 0.34236864
Iteration 1242, loss = 0.34234988
Iteration 1243, loss = 0.34233543
Iteration 1244, loss = 0.34231657
Iteration 1245, loss = 0.34229924
Iteration 1246, loss = 0.34228074
Iteration 1247, loss = 0.34226406
Iteration 1248, loss = 0.34224557
Iteration 1249, loss = 0.34222747
Iteration 1250, loss = 0.34220929
Iteration 1251, loss = 0.34219135
Iteration 1252, loss = 0.34217505
Iteration 1253, loss = 0.34215836
Iteration 1254, loss = 0.34214018
Iteration 1255, loss = 0.34212303
Iteration 1256, loss = 0.34210789
Iteration 1257, loss = 0.34208854
Iteration 1258, loss = 0.34207057
Iteration 1259, loss = 0.34205431
Iteration 1260, loss = 0.34203580
Iteration 1261, loss = 0.34201797
Iteration 1262, loss = 0.34200051
Iteration 1263, loss = 0.34198245
Iteration 1264, loss = 0.34196619
Iteration 1265, loss = 0.34194746
Iteration 1266, loss = 0.34193353
Iteration 1267, loss = 0.34191845
Iteration 1268, loss = 0.34189577
Iteration 1269, loss = 0.34187838
Iteration 1270

In [32]:
y_perdict = clf.predict(X_test)

In [33]:
confusion_matrix(Y_test, y_perdict)

array([[   0,   87,    0],
       [   0, 1800,    0],
       [   0,   78,    0]], dtype=int64)

In [ ]:
Y_test

In [ ]:
y_perdict = list(y_perdict)

In [ ]:
len(y_perdict)

In [ ]:
df_test = df_an.iloc[8000:].copy()
df_test.reset_index(drop=True, inplace=True)
# df_test["SIGNAL"] = y_perdict

In [ ]:
df_test.SIGNAL

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5.result.value_counts()

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])